In [ ]:
import pprint, json, math, os, sys
import fitz
import pandas as pd
from collections import defaultdict

dir_path = "C:\\Users\\Kaustubh.keny\\OneDrive - Cogencis Information Services Ltd\\Documents\\mywork-repo\\"
fund_path = "C:\\Users\\Kaustubh.keny\\OneDrive - Cogencis Information Services Ltd\\Documents\\Dec 24"

# dir_path = "C:\\Users\\rando\\OneDrive\\Documents\\mywork-repo"
# fund_path =  "C:\\Users\\rando\\OneDrive\\Documents\\Dec 24"
sys.path.append(os.path.abspath(dir_path))

from app.utils import Helper
from app.fund_regex import *

dry_path = r'\data\output\DryRun.pdf'
fin_path = r'\data\input\financial_indices.xlsx'
mutual_fund = Helper.get_fund_paths(fund_path)

In [3]:
def extract_clipped_data(input:str, pages:list, bboxes:list):
        
        document = fitz.open(input)
        final_list = []
    
        for pgn in pages:
            page = document[pgn]
            
            all_blocks = [] #store every data from bboxes
            
            for bbox in bboxes:
                blocks, seen_blocks = [], set()  #store unique blocks based on content and bbox
                
                page_blocks = page.get_text('dict', clip=bbox)['blocks']
                for block in page_blocks:
                    if block['type'] == 0 and 'lines' in block: #type 0 means text block
                        #hash_key
                        block_key = (tuple(block['bbox']), tuple(tuple(line['spans'][0]['text'] for line in block['lines'])))
                        if block_key not in seen_blocks:
                            seen_blocks.add(block_key)
                            blocks.append(block)

                sorted_blocks = sorted(blocks, key=lambda x: (x['bbox'][1], x['bbox'][0]))
                all_blocks.append(sorted_blocks)

            final_list.append({
                "pgn": pgn,
                "block": all_blocks #will be list[list,list,..]
            })

        document.close()
        return final_list

In [4]:
def extract_data_relative_line(path: str, line_x: float, side: str):
    doc = fitz.open(path)
    pages = doc.page_count

    final_list = []

    for pgn in range(pages):
        page = doc[pgn]

        blocks = page.get_text("dict")["blocks"]
        sorted_blocks = sorted(blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
        extracted_blocks = []

        # Keep track of blocks to avoid duplicates
        added_blocks = set()

        for block in sorted_blocks:
            block_id = id(block)  # Unique identifier for the block

            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    origin = span["origin"]
                    x0, _ = origin

                    # Check the side condition
                    if side == "left" and x0 < line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added
                    elif side == "right" and x0 > line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added

      
        final_list.append({
            "pgn": pgn,
            "blocks": extracted_blocks
        })

    doc.close()

    return final_list

def extract_spans(data):
    final_span = []
    for pgn,blocks in enumerate(data):
        spans = []
        print(f"___{pgn}___")
        for num,block in enumerate(blocks.get('blocks',[])):
            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    if num in range(3,10):
                        print(span['text'], span['size'])
        final_span.append(spans)
    return final_span

def get_proper_fund_names(path:str,pages:list):
        
    doc = fitz.open(path)
    final_fund_names = dict()
    
    for pgn in range(doc.page_count):
        text_all = ''
        if pgn in pages:
            # print(pgn)
            page = doc[pgn]            
            blocks = page.get_text("dict")['blocks']
            
            sorted_blocks = sorted(blocks,key=lambda k:(k['bbox'][1],k['bbox'][0]))
            for count,block in enumerate(sorted_blocks):
                for line in block.get("lines", []):
                    for span in line.get("spans", []):
                        text = span['text'].strip()
                        if count in range(0,1):
                            text_all+=f" {text}"
            print(text_all)
        pattern = r"MIRAE ASSET.*?\b(?:ETF|EOF|FOF|FTF|FUND)\b"
        if matches := re.findall(pattern, text_all.strip(), re.DOTALL):
            final_fund_names[pgn] = matches[0]
        else:
            final_fund_names[pgn] = ""
    return final_fund_names
  
def get_clipped_data(input:str, bboxes:list[set]):
    
        document = fitz.open(input)
        final_list = []
        
        for pgn in range(document.page_count):
            page = document[pgn]

            blocks = []
            for bbox in bboxes:
                blocks.extend(page.get_text('dict', clip = bbox)['blocks']) #get all blocks
            
            filtered_blocks = [block for block in blocks if block['type']== 0 and 'lines' in block]
            # sorted_blocks = sorted(filtered_blocks, key= lambda x: (x['bbox'][1], x['bbox'][0]))
            
            final_list.append({
            "pgn": pgn,
            "block": filtered_blocks
            })
            
            
        document.close()
        return final_list
    
def get_clipped_text(input:str, bboxes:list[set]):

    document = fitz.open(input)
    final_list = []
    
    for pgn in range(document.page_count):
        page = document[pgn]
        blocks = []
        for bbox in bboxes:
            blocks = page.get_text('text', clip = bbox).split('\n') #get all blocks
  
        final_list.append({
        "pgn": pgn,
        "block": blocks
        })   
    document.close()
    return final_list

In [18]:
sample_path  = mutual_fund['Aditya Birla Sun Life Mutual Fund']
pages = [12, 14,16]
bboxes = [(200, 50, 380, 812),(380, 50, 580, 812)]

In [29]:
df = extract_clipped_data(sample_path,pages,bboxes)

In [ ]:
final_dict = {}
for num,page in enumerate(df):
    all_blocks = []
    nested = page['block']
    # print(len(nested))
    for t,blocks in enumerate(nested):
        print(f"Block: {t}")
        bbox_blocks = []
        for block in blocks:
            seen_entries = set()
            if lines := block.get('lines'):
                for line in lines:
                    if spans:= line.get('spans'):
                        for span in spans:
                            entry = (round(span['size']), span['text'].strip(), span['color'], span['origin'], tuple(span['bbox']), span['font'])
                            if entry not in seen_entries:
                                seen_entries.add(entry)
                                bbox_blocks.append(entry)
                            
        all_blocks.append(bbox_blocks)
        # print(f"Appended Block: {t}")
    final_dict[num] = all_blocks
    
            

In [ ]:
stop_words = FundRegex().STOP_WORDS

for p, data in final_dict.items():
    
    all_cleaned_blocks = []
    for inner_block in data:
        inner_data = []
        for block in inner_block:
            size, text, color, origin, bbox,font = block
            if text not in stop_words:
                inner_data.append(block)
        all_cleaned_blocks.append(inner_block)
    
    all_processed_blocks = []
    for inner_block in all_cleaned_blocks:
        inner_data = []
        for block in inner_block:
            size, text, color, origin, bbox, font = block
            text = text.strip()
            # if round(size) in range(data_conditions[0][0], data_conditions[0][1]) and color in data_conditions[1] and font in data_conditions[3]:
            #     size = data_conditions[2]  # Update size
            inner_data.append(block)
        all_processed_blocks.append(inner_data)
        

all_grouped_blocks = []
for inner_block in all_processed_blocks:
    grouped_blocks = defaultdict(list)
    
    for block in inner_block:
        y_coord = round(block[3][1])  # Round y-coordinate
        size = block[0]
        grouped_blocks[(y_coord, size)].append(block)
    
    all_grouped_blocks.append(grouped_blocks)  # Store per inner list


all_combined_blocks = []
for grouped_blocks in all_grouped_blocks:  # Iterate per inner list
    combined_inner_list = []
    
    for key, group in grouped_blocks.items():
        if key[1] == data_conditions[2]:  # If size matches the target condition
            combined_text = " ".join(item[1] for item in group).strip()
            
            if combined_text:  # Ignore empty text
                size, color, origin, bbox, font = group[0][0], group[0][2], group[0][3], group[0][4], group[0][5]
                combined_inner_list.append([size, combined_text, color, origin, bbox, font])
        else:
            for item in group:
                combined_inner_list.append(item)

    all_combined_blocks.append(combined_inner_list)

In [33]:
pth = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\data\output\check.json"
with open(pth,'w') as file:
    json.dump(final_dict,file)

In [16]:
Helper.get_structure(df)

List of 2 items
  Dictionary with keys: ['pgn', 'block']
    int
    List of 2 items
      List of 88 items
        Dictionary with keys: ['number', 'type', 'bbox', 'lines']
          int
          int
          tuple
          List of 1 items
            Dictionary with keys: ['spans', 'wmode', 'dir', 'bbox']
              List of 1 items
                Dictionary with keys: ['size', 'flags', 'font', 'color', 'ascender', 'descender', 'text', 'origin', 'bbox']
                  float
                  int
                  str
                  int
                  float
                  float
                  str
                  tuple
                  tuple
              int
              tuple
              tuple


In [ ]:
{'number': 6, 'type': 0, 'bbox': (313.8583068847656, 62.622066497802734, 374.2023010253906, 72.68606567382812), 'lines': [{'spans': [{'size': 8.0, 'flags': 20, 'font': 'AnekLatin-Bold', 'color': -14475488, 'ascender': 1.0149999856948853, 'descender': -0.24300000071525574, 'text': 'Investment Style', 'origin': (313.8583068847656, 70.7420654296875), 'bbox': (313.8583068847656, 62.622066497802734, 374.2023010253906, 72.68606567382812)}], 'wmode': 0, 'dir': (1.0, 0.0), 'bbox': (313.8583068847656, 62.622066497802734, 374.2023010253906, 72.68606567382812)}]}


In [ ]:
lines = [
    ((220, 0), (220, 812)),# Vertical line
    #((420, 0), (420, 1000))
]

rectangles = [(0, 50, 200, 812),(200, 50, 380, 812),(380, 50, 580, 812)]
pages = [i for i in range(1,110)]
Helper.draw_lines_on_pdf(sample_path, lines, rectangles, pages, dir_path +dry_path)


Modified PDF saved to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\data\output\DryRun.pdf


In [3]:
json_folder = dir_path + r'\data\output'
common_keys = Helper.extract_common_keys(json_folder)

In [4]:
data = Helper.get_all_pdf_data(sample_path)

In [6]:
data[9]

{'pgn': 9,
 'blocks': [{'number': 47,
   'type': 0,
   'bbox': (36.27389907836914,
    67.16098022460938,
    298.8459167480469,
    101.55298614501953),
   'lines': [{'spans': [{'size': 24.0,
       'flags': 20,
       'font': 'Inter-Bold',
       'color': -12684359,
       'ascender': 1.1130000352859497,
       'descender': -0.3199999928474426,
       'text': 'Samco Multi Cap Fund',
       'origin': (36.27389907836914, 93.87298583984375),
       'bbox': (36.27389907836914,
        67.16098022460938,
        298.8459167480469,
        101.55298614501953)}],
     'wmode': 0,
     'dir': (1.0, 0.0),
     'bbox': (36.27389907836914,
      67.16098022460938,
      298.8459167480469,
      101.55298614501953)}]},
  {'number': 48,
   'type': 0,
   'bbox': (37.22850036621094,
    98.85289764404297,
    347.48779296875,
    109.98089599609375),
   'lines': [{'spans': [{'size': 8.0,
       'flags': 4,
       'font': 'Inter-Light',
       'color': -12303035,
       'ascender': 1.070999979972839